In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
import random

In [49]:
data = pd.read_csv("data.csv")

data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [50]:
LOOK_BACK = 30
PREDICT_DAY = 1

In [51]:
def create_dataset(dataset, look_back=LOOK_BACK):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:i+look_back, :])
        Y.append(dataset[i+look_back, :])
    return np.array(X), np.array(Y)

X, Y = create_dataset(scaled_data)
train_size = int(len(X) * 0.8)
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

In [52]:
data

,Unnamed: 0.1,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,,
2010-01-04,0,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.3960
2010-01-05,1,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.3850
2010-01-06,2,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.3790
2010-01-07,3,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.3680
2010-01-08,4,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.3750
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,3644,0.6978,1.1174,4.0507,1.3073,6.9954,6.6829,7.7874,71.45,109.47,4.1260
2019-12-28,3645,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.0000
2019-12-29,3646,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.0000


In [53]:
model = Sequential()
model.add(
    Dense(32,input_shape=(LOOK_BACK, data.shape[1]), activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(16, activation="relu"))

model.add(Dense(data.shape[1]))

model.compile(loss="mean_squared_error", optimizer="adam")

In [54]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 30, 32)            384       
                                                                 
 dense_46 (Dense)            (None, 30, 64)            2112      
                                                                 
 dropout_7 (Dropout)         (None, 30, 64)            0         
                                                                 
 dense_47 (Dense)            (None, 30, 128)           8320      
                                                                 
 dropout_8 (Dropout)         (None, 30, 128)           0         
                                                                 
 dense_48 (Dense)            (None, 30, 64)            8256      
                                                                 
 dropout_9 (Dropout)         (None, 30, 64)          

In [55]:
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=1)

Epoch 1/10
2894/2894 [==============================] - 3s 874us/step - loss: 0.1490
Epoch 2/10
2894/2894 [==============================] - 2s 840us/step - loss: 0.1371
Epoch 3/10
2894/2894 [==============================] - 2s 863us/step - loss: 0.1360
Epoch 4/10
2894/2894 [==============================] - 3s 876us/step - loss: 0.1349
Epoch 5/10
2894/2894 [==============================] - 3s 889us/step - loss: 0.1340
Epoch 6/10
2894/2894 [==============================] - 2s 852us/step - loss: 0.1342
Epoch 7/10
2894/2894 [==============================] - 3s 882us/step - loss: 0.1348
Epoch 8/10
2894/2894 [==============================] - 3s 938us/step - loss: 0.1345
Epoch 9/10
2894/2894 [==============================] - 3s 1ms/step - loss: 0.1342
Epoch 10/10
2894/2894 [==============================] - 6s 2ms/step - loss: 0.1337


In [56]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainY_original = scaler.inverse_transform(trainY)
testY_original = scaler.inverse_transform(testY)

23/23 [==============================] - 0s 2ms/step


In [57]:
testPredictions = []
for i in range(len(testX)):
    prediction = model.predict(testX[i:i+1])
    testPredictions.append(prediction[0])

testPredictions = np.array(testPredictions)


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


In [93]:
random_start_index = random.randint(0, len(X) - PREDICT_DAY)


random_input_sequence = X[random_start_index]
true_output = Y[random_start_index]


random_input_reshaped = np.reshape(random_input_sequence, (1, LOOK_BACK, data.shape[1]))

predicted_output = model.predict(random_input_reshaped)
print(predicted_output[0].transpose().shape)
print(true_output.shape)
predicted_output = predicted_output[0].transpose()
MSE = 0
MAE = 0
MAPE = 0
for i in range(30):
    l = true_output+predicted_output[:,i]
    MSE += np.sum(np.square(l))/11
    m = np.abs(true_output-predicted_output[:,i])
    MAE += np.mean(m)
    n = (m/true_output)
    MAPE += np.mean(n)
MSE /= 30
MAE /= 30
MAPE /= 30

print('Random Sequence Evaluation:')
print('MSE:', MSE)
print('MAE:', MAE)
print('MAPE:', MAPE)


1/1 [==============================] - 0s 15ms/step
(11, 30)
(11,)
Random Sequence Evaluation:
MSE: 1.586263733541835
MAE: 0.20259009094693045
MAPE: 0.2776675232669042
